参考：
- [知乎](https://zhuanlan.zhihu.com/p/27999355)
- [推荐系统遇上深度学习(三)--DeepFM模型理论和实践](https://www.jianshu.com/p/6f1c2643d31b)
- [tensorflow-DeepFM](https://github.com/ChenglongChen/tensorflow-DeepFM/tree/master/example)

DeepFM是一个集成了FM和DNN的神经网络框架，思路和Google的Wide&Deep相似，都包括wide和deep两部分。W&D模型的wide部分是广义线性模型，DeepFM的wide部分则是FM模型，两者的deep部分都是深度神经网络。DeepFM神经网络部分，隐含层的激活函数用ReLu和Tanh做信号非线性映射，Sigmoid函数做CTR预估的输出函数。

W&D模型的输入向量维度很大，因为wide部分的特征包括了手工提取的pairwise特征组合，大大提高计算复杂度。和W&D模型相比，DeepFM的wide和deep部分共享相同的输入，可以提高训练效率，不需要额外的特征工程，用FM建模low-order的特征组合，用DNN建模high-order的特征组合，因此可以同时从raw feature中学习到low-和high-order的feature interactions。在真实应用市场的数据和criteo的数据集上实验验证，DeepFM在CTR预估的计算效率和AUC
、LogLoss上超越了现有的模型（LR、FM、FNN、PNN、W&D）。

market发现，一些order-2 interaction（如app category和time-stamp）、order-3 interaction（user gender、age和app category）都可以作为CTR的signal。因此，low-order和high-order的feature interactions都可以在CTR预估中发挥作用。但如何有效构建interactions呢？大部分的特征组合隐藏在数据中，难以发现关联关系，只能通过机器学习来自动挖掘。

# 相关工作
FTRL算法（McMahan et al. 2013），generalized linear model虽然简单，但实践中很有效果。但这类线性模型难以学习组合特征，一般需要手动构建特征向量，难以处理高阶组合或者没有出现在训练数据中的组合。

Factorization Machine模型（Rendle
2010），对特征之间进行向量内积，实现特征们的逐对组合pairwise interactions。理论上FM可以对高阶特征组合建模，但实践中只用order-2特征因为其高复杂度。

DNN在特征表示学习中很有效果，可以用来学习组合特征。（Liu et al., 2015）和（Zhang et al., 2014）扩展CNN和RNN用于CTR预估，但CNN-based模型对领域特征有偏biased，RNN-based模型适合用在有时序依赖的点击数据。Factorization-machine supported Neural Network（FNN）（Zhang et al., 2016），在使用DNN前预训练FM，因此限制了FM的能力。Product-based Neural
Network（PNN）（Qu et al., 2016）在DNN的embedding层和全连接层之间引入product层，来研究feature interactions。

Wide & Deep模型(Cheng
et al., 2016)认为，PNN/FNN和其他Deep模型提取很少low-order的feature interactions，提出的W&D模型可以同时对low-order和high-order建模，但要对wide和deep部分模型分别输入，其中wide部分还需要人工特征工程。
![image.png](pic/DeepFM1.png)

# 模型实现

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score
from time import time
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
from yellowfin import YFOptimizer

In [6]:
class DeepFM(BaseEstimator, TransformerMixin):
    def __init__(self,feature_size,field_size, embedding_size = 8,
                 dropout_fm = [1,1],deep_layers = [32,32],
                 dropout_deep = [0.5,0.5,0.5],
                 deep_layers_activation = tf.nn.relu,
                 epoch = 10,batch_size = 256,
                 learning_rate = 0.001, optimizer_type = "adam",
                 batch_norm=0, batch_norm_decay=0.995,
                 verbose=False, random_seed=2016,
                 use_fm=True, use_deep=True,
                 loss_type="logloss", eval_metric=roc_auc_score,
                 l2_reg=0.0, greater_is_better=True):
        assert (use_fm or use_deep)
        assert loss_type in ["logloss", "mse"], \
            "loss_type can be either 'logloss' for classification task or 'mse' for regression task"

        self.feature_size = feature_size        # denote as M, size of the feature dictionary,one-hot之后的所有特征
        self.field_size = field_size            # denote as F, size of the feature fields，未one-hot的所有特征维度，
                                                # 对类别特征的每一个类别都做了一个因变量
        self.embedding_size = embedding_size    # denote as K, size of the feature embedding

        self.dropout_fm = dropout_fm
        self.deep_layers = deep_layers
        self.dropout_deep = dropout_deep
        self.deep_layers_activation = deep_layers_activation
        self.use_fm = use_fm
        self.use_deep = use_deep
        self.l2_reg = l2_reg

        self.epoch = epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.optimizer_type = optimizer_type

        self.batch_norm = batch_norm
        self.batch_norm_decay = batch_norm_decay

        self.verbose = verbose
        self.random_seed = random_seed
        self.loss_type = loss_type
        self.eval_metric = eval_metric
        self.greater_is_better = greater_is_better
        self.train_result, self.valid_result = [], []

        self._init_graph()

    def _init_graph(self):
        self.graph = tf.Graph()
        with self.graph.as_default():
            tf.set_random_seed(self.random_seed)

            # input
            # Xi: [[ind1_1, ind1_2, ...], [ind2_1, ind2_2, ...], ..., [indi_1, indi_2, ..., indi_j, ...], ...]
            #indi_j is the feature index of feature field j of sample i in the dataset
            # Xv: [[val1_1, val1_2, ...], [val2_1, val2_2, ...], ..., [vali_1, vali_2, ..., vali_j, ...], ...]
            # vali_j is the feature value of feature field j of sample i in the dataset
            # vali_j can be either binary (1/0, for binary/categorical features) or float (e.g., 10.24, for numerical features)
            self.feat_index = tf.placeholder(tf.int32,shape=[None,None],name='feat_index') # None * F
            self.feat_value = tf.placeholder(tf.float32,shape=[None,None],name='feat_value') # None * F
            self.label = tf.placeholder(tf.float32,shape=[None,1],name='label')

            self.dropout_keep_fm = tf.placeholder(tf.float32, shape=[None], name="dropout_keep_fm")
            self.dropout_keep_deep = tf.placeholder(tf.float32, shape=[None], name="dropout_keep_deep")

            self.train_phase = tf.placeholder(tf.bool,name='train_phase')

            # Variable
            self.weights = self._initialize_weights()

            # model
            self.embeddings = tf.nn.embedding_lookup(self.weights['feature_embeddings'],self.feat_index)
            feat_value = tf.reshape(self.feat_value,shape=[-1,self.field_size,1])
            self.embeddings = tf.multiply(self.embeddings,feat_value) # None*F*K


            # first-order term
            self.y_first_order = tf.nn.embedding_lookup(self.weights["feature_bias"],self.feat_index) # None*F*1
            self.y_first_order = tf.reduce_sum(tf.multiply(self.y_first_order,feat_value),2)# None*G
            self.y_first_order = tf.nn.dropout(self.y_first_order,self.dropout_fm[0])


            # second-order term
            # sum_square part
            self.summed_features_emb = tf.reduce_sum(self.embeddings, 1)  # None * K
            self.summed_features_emb_square = tf.square(self.summed_features_emb)  # None * K

            # square_sum part
            self.squared_features_emb = tf.square(self.embeddings)
            self.squared_sum_features_emb = tf.reduce_sum(self.squared_features_emb, 1)  # None * K

            # second order
            # 0.5*sum((sum(vx))**2-sum(v**2*x**2))
            self.y_second_order = 0.5 * tf.subtract(self.summed_features_emb_square, self.squared_sum_features_emb)  # None * K
            self.y_second_order = tf.nn.dropout(self.y_second_order, self.dropout_keep_fm[1])  # None * K

            # 与fm不同，first_order 和second_order是通过concat联在一起的
            # self.fm = tf.add(self.y_first_order,self.y_second_order)

            # Deep
            self.y_deep = tf.reshape(self.embeddings,shape=[-1,self.field_size*self.embedding_size]) # None*(F*K)
            # embedding dropout
            self.y_deep = tf.nn.dropout(self.y_deep,self.dropout_keep_deep[0])
            for i in range(0,len(self.deep_layers)):
                self.y_deep = tf.add(tf.matmul(self.y_deep,self.weights['layer_%d'%i]),self.weights['bias_%d' %i])# None * layer[i] * 1
                if self.batch_norm:
                    self.y_deep = self.batch_norm_layer(self.y_deep,train_phase=self.train_phase,scope_bn="bn_%d" %i) # None * layer[i] * 1
                self.y_deep = self.deep_layers_activation(self.y_deep)
                self.y_deep = tf.nn.dropout(self.y_deep,self.dropout_keep_deep[i+1])

            # DeepFM
            if self.use_fm and self.use_deep:
                input = tf.concat([self.y_first_order,self.y_second_order,self.y_deep],axis=1)
            elif self.use_fm:
                input = tf.concat([self.y_first_order,self.y_second_order],axis=1)
            elif self.use_deep:
                input = self.y_deep
            self.out = tf.add(tf.matmul(input,self.weights['concat_projection']),self.weights["concat_bias"])

            # loss
            if self.loss_type == "logloss":
                self.out = tf.nn.sigmoid(self.out)
                self.loss = tf.losses.log_loss(self.label,self.out)
            elif self.loss_type == 'mse':
                self.loss = tf.nn.l2_loss(tf.subtract(self.label,self.out))

            # l2
            if self.l2_reg > 0:
                self.loss += tf.contrib.layers.l2_regularizer(self.l2_reg)(self.weights["concat_projection"])
                if self.use_deep:
                    for i in range(len(self.deep_layers)):
                        self.loss += tf.contrib.layers.l2_regularizer(
                            self.l2_reg)(self.weights["layer_%d"%i])

            # optimizer
            if self.optimizer_type == 'adam':
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate,
                                                        beta1=0.9,
                                                        beta2=0.999,
                                                        epsilon=1e-8).minimize(self.loss)
            elif self.optimizer_type == 'adagrad':
                self.optimizer = tf.train.AdagradOptimizer(learning_rate=self.learning_rate,
                                                           initial_accumulator_value=1e-8).minimize(self.loss)
            elif self.optimizer_type == 'gd':
                self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate,
                                                                   ).minimize(self.loss)
            elif self.optimizer_type =='momentum':
                self.optimizer = tf.train.MomentumOptimizer(learning_rate=self.learning_rate,
                                                            momentum=0.95).minimize(self.loss)
            elif self.optimizer_type == 'yellowfin':
                self.optimizer = YFOptimizer(learning_rate=self.learning_rate,momentum=0.0).minimize(self.loss)

            # init
            self.saver = tf.train.Saver()
            init = tf.global_variables_initializer()
            self.sess = self._init_session()
            self.sess.run(init)

            # number of params
            total_parameters = 0
            for variable in self.weights.values():
                shape = variable.get_shape()
                variable_parameters = 1
                for dim in shape:
                    variable_parameters *= dim.value
                total_parameters += variable_parameters
            if self.verbose > 0:
                print("#params: %d" % total_parameters)

    def _init_session(self):
        #config = tf.ConfigProto(device_count={"gpu":0})
        #config.gpu_options.allow_growth = True
        return  tf.Session()

    def _initialize_weights(self):
        weights = dict()

        # embeddings
        weights["feature_embeddings"] = tf.Variable(tf.random_normal([self.feature_size,self.embedding_size],0,0.01),
                                                    name='feature_embeddings') # feature_embedding: f*K

        weights["feature_bias"] = tf.Variable(tf.random_uniform([self.feature_size,1],0.0,1.0),
                                              name='feature_bias')


        # deep layers
        # number of layer of hidden dnn
        num_layer = len(self.deep_layers)
        input_size = self.field_size * self.embedding_size

        #Glorot and Bengio (2010) 建议使用标准初始化（normalized initialization）
        glorot = np.sqrt(2.0 / (input_size+self.deep_layers[0]))
        weights["layer_0"] = tf.Varible(np.random.normal(loc=0,scale=glorot,
                                                         size=(input_size,self.deep_layers[0])),dtype=np.float32)
        weights['bias_0'] = tf.Variable(np.random.normal(loc=0,scale=glorot,
                                                         size=(1,self.deep_layers[0])),dtype=tf.float32)

        for i in range(1,num_layer):
            glorot = np.sqrt(2.0 / (self.deep_layers[i-1]+self.deep_layers[i]))
            weights['layer_%d'%i] = tf.Varible(np.random.normal(loc=0,scale=glorot,
                                                             size=(self.deep_layers[i-1],self.deep_layers[i])),dtype=np.float32)
            weights['bias_%d' %i] = tf.Variable(np.random.normal(loc=0,scale=glorot,
                                                             size=(1,self.deep_layers[i])),dtype=tf.float32)


        # final concat projection layer
        if self.use_fm and self.use_deep:
            input_size = self.field_size + self.embedding_size + self.deep_layers[-1]
        elif self.use_fm:
            input_size = self.field_size + self.embedding_size
        elif self.use_deep:
            input_size = self.deep_layers[-1]

        weights["concat_projection"] = tf.Variable(
            np.random.normal(loc=0, scale=glorot, size=(input_size, 1)),
            dtype=np.float32)  # layers[i-1]*layers[i]
        weights["concat_bias"] = tf.Variable(tf.constant(0.01), dtype=np.float32)

        return weights

    def batch_norm_layer(self,x,train_phase,scope_bn):
        '''
        批标准化所要解决的问题是：模型参数在学习阶段的变化，会使每个隐藏层输出的分布也发生改变。这意味着靠后的层要在训练过程中去适应这些变化。
        为了解决这个问题，论文BN2015提出了批标准化，即在训练时作用于每个神经元激活函数（比如sigmoid或者ReLU函数）的输入，
        使得基于每个批次的训练样本，激活函数的输入都能满足均值为0，方差为1的分布。
        :param x:
        :param train_phase:
        :param scope_bn:
        :return:
        '''
        bn_train = batch_norm(x, decay=self.batch_norm_decay, center=True, scale=True, updates_collections=None,
                              is_training=True, reuse=None, trainable=True, scope=scope_bn)
        bn_inference = batch_norm(x, decay=self.batch_norm_decay, center=True, scale=True, updates_collections=None,
                                  is_training=False, reuse=True, trainable=True, scope=scope_bn)

        #tf.cond()类似于c语言中的if...else...，用来控制数据流向，但是仅仅类似而已，其中差别还是挺大的。关于tf.cond（）函数的具体操作，我参考了tf的说明文档。
        # format：tf.cond(pred, fn1, fn2, name=None)
        z = tf.cond(train_phase, lambda: bn_train, lambda: bn_inference)
        return z

    def get_batch(self,Xi,Xv,y,batch_size,index):
        start = index * batch_size
        end = (index+1)*batch_size
        end = end if end<len(y) else len(y)
        return Xi[start:end],Xv[start:end],[[y_] for y_ in y[start:end]]

    # shuffle three lists simutaneously
    def shuffle_in_unison_scary(self,a,b,c):
        rng_state = np.random.get_state()
        np.random.shuffle(a)
        np.ranfom.set_state(rng_state)
        np.random.shuffle(b)
        np.ranfom.set_state(rng_state)
        np.random.shuffle(c)

    def fit_on_batch(self, Xi, Xv, y):
        feed_dict = {self.feat_index: Xi,
                     self.feat_value: Xv,
                     self.label: y,
                     self.dropout_keep_fm: self.dropout_fm,
                     self.dropout_keep_deep: self.dropout_deep,
                     self.train_phase: True}
        loss, opt = self.sess.run((self.loss, self.optimizer), feed_dict=feed_dict)
        return loss

    def fit(self, Xi_train, Xv_train, y_train,
            Xi_valid=None, Xv_valid=None, y_valid=None,
            early_stopping=False, refit=False):
        """
        :param Xi_train: [[ind1_1, ind1_2, ...], [ind2_1, ind2_2, ...], ..., [indi_1, indi_2, ..., indi_j, ...], ...]
                         indi_j is the feature index of feature field j of sample i in the training set
        :param Xv_train: [[val1_1, val1_2, ...], [val2_1, val2_2, ...], ..., [vali_1, vali_2, ..., vali_j, ...], ...]
                         vali_j is the feature value of feature field j of sample i in the training set
                         vali_j can be either binary (1/0, for binary/categorical features) or float (e.g., 10.24, for numerical features)
        :param y_train: label of each sample in the training set
        :param Xi_valid: list of list of feature indices of each sample in the validation set
        :param Xv_valid: list of list of feature values of each sample in the validation set
        :param y_valid: label of each sample in the validation set
        :param early_stopping: perform early stopping or not
        :param refit: refit the model on the train+valid dataset or not
        :return: None
        """
        has_valid = Xv_valid is not None
        for epoch in range(self.epoch):
            t1= time()
            self.shuffle_in_unison_scary(Xi_train,Xv_train,y_train)
            total_batch = int(len(y_train)/self.batch_size)
            for i in range(total_batch):
                Xi_batch, Xv_batch, y_batch = self.get_batch(Xi_train, Xv_train, y_train, self.batch_size, i)
                self.fit_on_batch(Xi_batch, Xv_batch, y_batch)

            # evaluate training and validation datasets in this epoch
            train_result = self.evaluate(Xi_train,Xv_train,y_train)
            self.train_result.append(train_result)

            if has_valid:
                valid_result = self.evaluate(Xi_train,Xv_train,y_train)
                self.valid_result.append(train_result)

            if self.verbose > 0 and epoch % self.verbose == 0:
                if has_valid:
                    print("[%d] train-result=%.4f, valid-result=%.4f [%.1f s]"
                          % (epoch + 1, train_result, valid_result, time() - t1))
                else:
                    print("[%d] train-result=%.4f [%.1f s]"
                          % (epoch + 1, train_result, time() - t1))
            if has_valid and early_stopping and self.training_termination(self.valid_result):
                break

        # fit a few more epoch on train+valid until result reaches the best_train_score
        if has_valid and refit:
            if self.greater_is_better:
                best_valid_score = max(self.valid_result)
            else:
                best_valid_score = min(self.valid_result)
            best_epoch = self.valid_result.index(best_valid_score)
            best_train_score = self.train_result[best_epoch]
            Xi_train = Xi_train + Xi_valid
            Xv_train = Xv_train + Xv_valid
            y_train = y_train + y_valid
            for epoch in range(100):
                self.shuffle_in_unison_scary(Xi_train, Xv_train, y_train)
                total_batch = int(len(y_train) / self.batch_size)
                for i in range(total_batch):
                    Xi_batch, Xv_batch, y_batch = self.get_batch(Xi_train, Xv_train, y_train,
                                                                 self.batch_size, i)
                    self.fit_on_batch(Xi_batch, Xv_batch, y_batch)
                # check
                train_result = self.evaluate(Xi_train, Xv_train, y_train)
                if abs(train_result - best_train_score) < 0.001 or \
                        (self.greater_is_better and train_result > best_train_score) or \
                        ((not self.greater_is_better) and train_result < best_train_score):
                    break





    def predict(self,Xi,Xv):
        """
       :param Xi: list of list of feature indices of each sample in the dataset
       :param Xv: list of list of feature values of each sample in the dataset
       :return: predicted probability of each sample
       """
        # dummy y
        dummy_y = [1] * len(Xi)
        batch_index = 0
        Xi_batch, Xv_batch, y_batch = self.get_batch(Xi, Xv, dummy_y, self.batch_size, batch_index)
        y_pred = None
        while len(Xi_batch) > 0:
            num_batch = len(y_batch)
            feed_dict = {self.feat_index: Xi_batch,
                         self.feat_value: Xv_batch,
                         self.label: y_batch,
                         self.dropout_keep_fm: [1.0] * len(self.dropout_fm),
                         self.dropout_keep_deep: [1.0] * len(self.dropout_deep),
                         self.train_phase: False}
            batch_out = self.sess.run(self.out, feed_dict=feed_dict)
    
            if batch_index == 0:
                y_pred = np.reshape(batch_out, (num_batch,))
            else:
                y_pred = np.concatenate((y_pred, np.reshape(batch_out, (num_batch,))))
    
            batch_index += 1
            Xi_batch, Xv_batch, y_batch = self.get_batch(Xi, Xv, dummy_y, self.batch_size, batch_index)
    
        return y_pred


    def training_termination(self, valid_result):
        if len(valid_result) > 5:
            if self.greater_is_better:
                if valid_result[-1] < valid_result[-2] and \
                        valid_result[-2] < valid_result[-3] and \
                        valid_result[-3] < valid_result[-4] and \
                        valid_result[-4] < valid_result[-5]:
                    return True
            else:
                if valid_result[-1] > valid_result[-2] and \
                        valid_result[-2] > valid_result[-3] and \
                        valid_result[-3] > valid_result[-4] and \
                        valid_result[-4] > valid_result[-5]:
                    return True
        return False

    def evaluate(self, Xi, Xv, y):
        """
        :param Xi: list of list of feature indices of each sample in the dataset
        :param Xv: list of list of feature values of each sample in the dataset
        :param y: label of each sample in the dataset
        :return: metric of the evaluation
        """
        y_pred = self.predict(Xi, Xv)
        return self.eval_metric(y, y_pred)
